Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') > DESIRED_ACCURACY):
      print('\n Reached 99.9% accuracy so cancelling training!')
      self.model.stop_training = True

callbacks = myCallback()

--2020-08-09 16:48:52--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.28.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.28.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2020-08-09 16:48:52 (102 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [9]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [10]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s/', 
    target_size=(150,150), 
    batch_size=10, 
    class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [11]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,  
    epochs=20,
    verbose=1, 
    callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
8/8 [==============================] - 2s 223ms/step - loss: 1.8382 - acc: 0.5500
Epoch 2/20
8/8 [==============================] - 2s 224ms/step - loss: 0.6551 - acc: 0.6000
Epoch 3/20
8/8 [==============================] - 2s 223ms/step - loss: 0.5058 - acc: 0.9125
Epoch 4/20
8/8 [==============================] - 2s 225ms/step - loss: 0.2675 - acc: 0.8875
Epoch 5/20
8/8 [==============================] - 2s 224ms/step - loss: 0.3069 - acc: 0.8250
Epoch 6/20
8/8 [==============================] - 2s 224ms/step - loss: 0.1474 - acc: 0.9375
Epoch 7/20
8/8 [==============================] - 2s 224ms/step - loss: 0.1097 - acc: 0.9750
Epoch 8/20
8/8 [==============================] - 2s 221ms/step - loss: 0.0875 - acc: 0.9625
Epoch 9/20
8/8 [==============================] - 2s 224ms/step - loss: 0.0778 - acc: 0.9625
Epoch 10/20
8/8 [==============================] - ETA: 0s - loss: 0.0433 - acc: 1.0000
 Reached 99.9% accuracy so cancelling training!
8/8 [======================